### 1. 키워드파일 업로드

In [31]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# CSV 파일에서 클러스터링 결과 로드
df_inheritance = pd.read_csv(r'C:\Users\user\Documents\no-f\241030\df_inheritance_keyword.csv')


In [33]:
df_inheritance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4162 entries, 0 to 4161
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례정보일련번호       4162 non-null   int64 
 1   사건번호           4162 non-null   object
 2   선고             4162 non-null   object
 3   판결유형           4162 non-null   object
 4   판시사항           3248 non-null   object
 5   판결요지           2930 non-null   object
 6   참조조문           3221 non-null   object
 7   참조판례           1933 non-null   object
 8   판례내용           4162 non-null   object
 9   판례내용_상단        4162 non-null   object
 10  판례내용_이유        4162 non-null   object
 11  판례내용_이유_전처리    4142 non-null   object
 12  판례내용_이유_불용어제거  4127 non-null   object
 13  클러스터           4162 non-null   int64 
 14  키워드            4162 non-null   object
dtypes: int64(2), object(13)
memory usage: 487.9+ KB


### 2. kobert 불러와서 임베딩하기

In [48]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# 원본 데이터프레임 로드
df_inheritance = pd.read_csv('df_inheritance_keyword.csv')

# 판례 내용과 클러스터 키워드 임베딩 생성
df_inheritance['내용임베딩'] = df_inheritance['판례내용'].apply(lambda x: encode_sentence(x) if pd.notnull(x) else None)
df_inheritance['키워드임베딩'] = df_inheritance['키워드'].apply(lambda x: encode_sentence(x) if pd.notnull(x) else None)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [49]:
# 사용자 입력
user_keyword = "세법"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 유사도 계산 및 사건명 추출
similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['키워드임베딩'] is not None:
        # 키워드 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        similarity = cosine_similarity(user_keyword_embedding, row['키워드임베딩']).flatten()[0]
        similarity_scores.append((row['사건번호'], row['판례내용'], similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_scores = sorted(similarity_scores, key=lambda x: x[2], reverse=True)
top_n = sorted_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명:")
for 사건번호, 판례내용, score in top_n:
    print(f"사건번호: {사건번호}, 유사도: {score:.4f}, 판례내용: {판례내용}")

사용자 키워드와 관련된 사건명:
사건번호: 2017두62716, 유사도: 0.4102, 판례내용: 【원고, 상고인】 원고 1 외 3인 (소송대리인 변호사 정병문 외 3인)【피고, 피상고인】 강남세무서장 외 1인 (소송대리인 법무법인(유한) 태평양 담당변호사 김승호 외 3인)【원심판결】 서울고법 2017. 8. 11. 선고 2017누30360 판결【주    문】상고를 모두 기각한다.  상고비용은 원고들이 부담한다.【이    유】  상고이유를 판단한다.  1. 사안의 개요   가. 망 소외 1(이하 ‘피상속인’이라고 한다)은 1993. 4. 20. 홍콩에서 완구사업을 목적으로 하는 Fair Trade Ltd.(이하 ‘FT’라 한다)를 설립하여 운영하였다. 2003. 10. 15.에는 홍콩에서 Grandfair Trading Ltd.(이하 ‘GFT’라 한다)가 설립되어 이사로 등재된 피상속인의 아들 원고 1과 그 이종사촌 소외 2가 총 발행주식 2주를 인수하였다(이하 FT와 GFT를 통틀어 ‘이 사건 홍콩법인’이라고 한다). 한편 FT는 2004. 4. 30. 그 사업과 자산 일체를 GFT에 양도하였다.  나. 피상속인은 2005. 3. 15. 사망하였고, 당시 UBS, WINCHANG BANK, CAYMAN BANK 등 3개 은행에 피상속인 및 그 형 소외 3의 공동 명의로 각 금융계좌가 개설되어 있었는데 각 계좌에는 합계 미화 12,858,689달러(이하 ‘이 사건 해외자산’이라고 한다)가 보관되어 있었다.  다. 원고들은 피상속인의 공동상속인들로서 상속세를 신고ㆍ납부하면서, 이 사건 홍콩법인의 비상장주식과 이 사건 해외자산을 상속재산에 포함시키지 아니하였다.  라. 이에 피고 강남세무서장은 이 사건 해외자산을 상속재산에 포함시켜 2014. 3. 10. 원고들에게 상속세 10,121,885,900원을 부과ㆍ고지하였다. 그런 다음 다시 이 사건 홍콩법인의 비상장주식도 상속재산에 포함된다고 보고, 2014. 7. 1. 원고들에게 상속세 합계 16,632,980,40

### 2. 사건명가져오기 위해서 판례목록 데이터 불러오기

In [59]:
# 사건 리스트 로드
inheritance_list = pd.read_csv('inheritance_list.csv')

# 사건명 추가를 위해 df_inheritance 데이터프레임과 병합
df_inheritance = df_inheritance.merge(inheritance_list[['사건번호', '사건명']], on='사건번호', how='left')

In [68]:
df_inheritance.head(2)

,판례정보일련번호,사건번호,선고,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유,판례내용_이유_전처리,판례내용_이유_불용어제거,클러스터,키워드,내용임베딩,키워드임베딩,사건명
0,238913,2023다221144,선고,판결,"[1] 가분채권이 공동상속되는 경우의 법률관계 / 주식이 공동상속되는 경우, 공동...",[1] 금전채권과 같이 급부의 내용이 가분인 채권은 공동상속되는 경우 상속개시와 ...,"[1] 민법 제269조, 제278조, 제547조 제1항, 제1005조, 제1006...","[1] 대법원 2003. 5. 30. 선고 2003다7074 판결, 대법원 200...","【원고, 상고인】 원고 (소송대리인 법무법인 참진 담당변호사 이영동)【피고, 피상고...","【원고, 상고인】 원고 (소송대리인 법무법인 참진 담당변호사 이영동)【피고, 피상고...",【이 유】 상고이유를 판단한다. 1. 사안의 개요 가. 사실관계 원...,상고이유를 판단한다. 1. 사안의 개요 가. 사실관계 원심판결 이유와 기록에 따르면...,개요 사실관계 망인 은행 예금 채권 은행 판매 투자신탁 형태 MMF Money Ma...,2,"신청, 상속, 부동산, 주택, 사망, 건물, 계약, 회사, 재산, 호주","[[0.12730557, -0.25995082, 0.6528336, -3.01953...","[[0.08193795, 0.05260368, 0.30794698, -3.76139...",예금[투자신탁 형태 단기금융집합투자기구(MMF) 수익권의 공동상속 효과가 문제된 사건]
1,240673,2021두52143,선고,판결,대한민국에서 주로 체류하며 대한민국에 있는 건축사무소 등에서 건축설계사로 근무하...,NaN,구 섭외사법(2001. 4. 7. 법률 제6465호 국제사법으로 전부 개정되기 전의...,NaN,"【원고, 상고인】 원고 1 외 3인 (소송대리인 변호사 이종건)【피고, 피상고인】 ...","【원고, 상고인】 원고 1 외 3인 (소송대리인 변호사 이종건)【피고, 피상고인】 ...","【이 유】 상고이유를 판단한다. 1. 원심판결 이유와 기록에 따르면, 다...","상고이유를 판단한다. 1. 원심판결 이유와 기록에 따르면, 다음의 사실을 알 수 있...",인과 대한민국 혼인신고 마쳤고 자녀 미합중국 캘리포니아 출생 미합중국 시민권 취...,2,"신청, 상속, 부동산, 주택, 사망, 건물, 계약, 회사, 재산, 호주","[[0.14671206, -0.27190125, 0.6653236, -2.92998...","[[0.08193795, 0.05260368, 0.30794698, -3.76139...",상속세부과처분취소


### 3. 유사도 계산해서 추천사건명 출력하기

#### - 사용자가 입력한 키워드와 키워드임베딩, 내용임베딩 유사도 평균계산 추천하기

In [62]:
# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 유사도 계산 및 사건명 추출
similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['키워드임베딩'] is not None and row['내용임베딩'] is not None:
        # 키워드 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        keyword_similarity = cosine_similarity(user_keyword_embedding, row['키워드임베딩']).flatten()[0]
        
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['내용임베딩']).flatten()[0]
        
        # 두 유사도의 평균을 계산
        average_similarity = (keyword_similarity + content_similarity) / 2
        
        # 결과 저장
        similarity_scores.append((row['사건번호'], average_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
top_n = sorted_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명:")
for 사건번호, score in top_n:
    # 사건번호를 통해 사건명 조회
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명:
사건번호: 2005도5338, 사건명: 특정경제범죄가중처벌등에관한법률위반(횡령)(인정된죄명:업무상횡령)·업무상횡령, 유사도: 0.4294
사건번호: 93타경23863, 사건명: 부동산임의경매, 유사도: 0.4245
사건번호: 2009브23, 사건명: 상속한정승인결정에대한즉시항고, 유사도: 0.4243
사건번호: 2020가소16553, 사건명: 예금반환, 유사도: 0.4230
사건번호: 2007카기134, 사건명: 위헌법률심판제청, 유사도: 0.4226
사건번호: 99다45864, 사건명: 소유권이전등기, 유사도: 0.4218
사건번호: 2018두32927, 사건명: 취득세등경정청구거부처분취소, 유사도: 0.4218
사건번호: 79다1009, 사건명: 소유권보존등기말소, 유사도: 0.4213
사건번호: 2011도4645, 사건명: 특허법 위반, 유사도: 0.4212
사건번호: 68다1587, 사건명: 소유권이전등기말소, 유사도: 0.4197


#### - 사용자가 입력한 키워드와 키워드임베딩 유사도로 추천하기

In [65]:
# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 키워드 임베딩 유사도 계산 및 사건명 추출
keyword_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['키워드임베딩'] is not None:
        # 키워드 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        keyword_similarity = cosine_similarity(user_keyword_embedding, row['키워드임베딩']).flatten()[0]
        keyword_similarity_scores.append((row['사건번호'], keyword_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_keyword_scores = sorted(keyword_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_keywords = sorted_keyword_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 (키워드 임베딩):")
for 사건번호, score in top_n_keywords:
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 (키워드 임베딩):
사건번호: 2023다221144, 사건명: 예금[투자신탁 형태 단기금융집합투자기구(MMF) 수익권의 공동상속 효과가 문제된 사건], 유사도: 0.4670
사건번호: 2021두52143, 사건명: 상속세부과처분취소, 유사도: 0.4670
사건번호: 2022누67403, 사건명: 상속세부과처분취소, 유사도: 0.4670
사건번호: 2022스625, 사건명: 상속재산관리인선임, 유사도: 0.4670
사건번호: 2021다294674, 사건명: 예금반환 [망인의 공동상속인 중 1인인 원고가 은행인 피고를 상대로 망인의 청약저축예금 반환을 구하는 사안], 유사도: 0.4670
사건번호: 2019나2050091, 사건명: 상속회복청구등의소, 유사도: 0.4670
사건번호: 2019구합83052, 사건명: 상속세경정거부처분취소, 유사도: 0.4670
사건번호: 2017스516, 517, 사건명: 사건명 없음, 유사도: 0.4670
사건번호: 2016브5(본심판), 2016브6(반심판), 2016브7(반심판), 2016브8(병합), 사건명: 사건명 없음, 유사도: 0.4670
사건번호: 2015브102(본심판), 2015브103(반심판), 사건명: 사건명 없음, 유사도: 0.4670


#### - 사용자가 입력한 키워드와 내용 유사도로 추천하기

In [67]:
# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['내용임베딩'] is not None:
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['내용임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 (판례내용 임베딩):")
for 사건번호, score in top_n_contents:
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 (판례내용 임베딩):
사건번호: 2006다46346, 사건명: 소유권이전등기등, 유사도: 0.4024
사건번호: 2008즈기1, 사건명: 기여분결정의심판청구, 유사도: 0.3967
사건번호: 2005도5338, 사건명: 특정경제범죄가중처벌등에관한법률위반(횡령)(인정된죄명:업무상횡령)·업무상횡령, 유사도: 0.3918
사건번호: 81다카1061, 사건명: 부당이득반환, 유사도: 0.3864
사건번호: 66나1654, 사건명: 부동산소유권이전등기말소등기절차이행청구사건, 유사도: 0.3862
사건번호: 85도2189, 사건명: 사기,공정증서원본불실기재,동행사, 유사도: 0.3858
사건번호: 86도2520, 사건명: 부동산소유권이전등기등에관한특별조치법위반, 유사도: 0.3844
사건번호: 98마40, 사건명: 등기공무원처분에대한이의, 유사도: 0.3830
사건번호: 93타경23863, 사건명: 부동산임의경매, 유사도: 0.3819
사건번호: 2009브23, 사건명: 상속한정승인결정에대한즉시항고, 유사도: 0.3816


# 망한거

In [56]:
# 판례내용_이유_불용어제거 열에서 null 값을 가진 행 제거
df_inheritance = df_inheritance.dropna(subset=['판례내용_이유_불용어제거'])

In [40]:

# 3. 문장 인코딩 함수
def encode_sentences(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 문장 임베딩

# 4. 클러스터와 키워드별 참조조문 임베딩 생성
embeddings_list = []
keyword_labels_list = []

# 데이터프레임을 순회하며 클러스터와 키워드별로 참조조문 임베딩
for _, row in df_inheritance.iterrows():
    # 판결요지 또는 참조조문을 사용할 수 있음
    if pd.notnull(row['참조조문']):  # 참조조문이 있는 경우
        reference = row['참조조문']
        cluster = row['클러스터']
        keyword = row['키워드']
        case_id = row['판례정보일련번호']  # 판례정보일련번호 추가
        
        # 참조조문 임베딩
        embedding = encode_sentences([reference])
        
        # 클러스터와 키워드 정보와 함께 저장
        embeddings_list.append(embedding.numpy()[0])  # 첫 번째 차원만 추가
        keyword_labels_list.append((case_id, cluster, keyword))  # 판례정보일련번호도 추가

# 5. 결과를 DataFrame으로 변환
embedding_df = pd.DataFrame(embeddings_list)
embedding_df['판례정보일련번호'] = [label[0] for label in keyword_labels_list]
embedding_df['클러스터'] = [label[1] for label in keyword_labels_list]
embedding_df['키워드'] = [label[2] for label in keyword_labels_list]

# 최종 결과 CSV로 저장
embedding_df.to_csv('keyword_reference_embeddings.csv', index=False, encoding='utf-8-sig')

print("키워드 및 클러스터별 참조조문 임베딩이 'keyword_reference_embeddings.csv' 파일에 저장되었습니다.")


키워드 및 클러스터별 참조조문 임베딩이 'keyword_reference_embeddings.csv' 파일에 저장되었습니다.


In [41]:
# 1. DB에서 클러스터 임베딩 로드 (CSV 파일을 사용하여 시뮬레이션)
embedding_df = pd.read_csv('keyword_reference_embeddings.csv')  # 수정된 파일명

In [42]:
print(embedding_df.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '761', '762', '763', '764', '765', '766', '767', '판례정보일련번호', '클러스터',
       '키워드'],
      dtype='object', length=771)


In [45]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# 1. 원본 데이터프레임 로드
df_inheritance = pd.read_csv(r'C:\Users\user\Documents\no-f\241030\df_inheritance_keyword.csv')

# 2. DB에서 클러스터 임베딩 로드
embedding_df = pd.read_csv('keyword_reference_embeddings.csv')

# 3. KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 4. 문장 인코딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# 5. 사용자 입력 문장
user_input = "세법"
user_embedding = encode_sentence(user_input)

# 6. DB에서 모든 문서 임베딩 가져오기
# '판례정보일련번호'가 포함된 임베딩을 가져옵니다.
all_embeddings = embedding_df.drop(columns=['클러스터', '키워드']).values  # 클러스터, 키워드 제외
all_embeddings_tensor = torch.tensor(all_embeddings)

# 7. 유사도 계산
similarity_scores = cosine_similarity(user_embedding.numpy(), all_embeddings_tensor.numpy()).flatten()

# 8. 유사도에 따라 문서 정렬
embedding_df['similarity'] = similarity_scores
sorted_documents = embedding_df.sort_values(by='similarity', ascending=False)

# 9. 가장 유사한 문서 상위 N개 선택
top_n = 10
recommended_documents = sorted_documents.head(top_n)

# 10. 추천 문서 출력
print("세법과 관련된 문서:")
for idx, row in recommended_documents.iterrows():
    # 판례정보일련번호가 embedding_df에 포함되어 있어야 합니다.
    reference_row = df_inheritance[df_inheritance['판례정보일련번호'] == row['판례정보일련번호']]
    if not reference_row.empty:
        reference = reference_row.iloc[0]['참조조문']
        print(f"클러스터: {row['클러스터']}, 유사도: {row['similarity']:.4f}, 내용: {reference}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 768 while Y.shape[1] == 769

In [15]:
# 문장 인코딩 함수
def encode_sentences(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 문장 임베딩

# 클러스터 라벨 생성
cluster_labels = df_inheritance['클러스터'].unique()  # 유일한 클러스터 라벨 추출


In [18]:
# 각 클러스터에 대한 문서 임베딩 생성
cluster_embeddings = {}
for label in cluster_labels:
    # 해당 클러스터의 문서 선택
    cluster_docs = df_inheritance[df_inheritance['클러스터'] == label]['참조조문'].dropna().tolist()
    cluster_docs = [doc for doc in cluster_docs if isinstance(doc, str) and doc.strip()]  # 빈 문자열과 None 제거
    
    # 빈 문서 체크
    if cluster_docs:  # 리스트가 비어 있지 않을 경우
        embeddings = encode_sentences(cluster_docs)
        cluster_embeddings[label] = embeddings
    else:
        print(f"클러스터 {label}에 문서가 없습니다.")

In [19]:
# 클러스터 라벨과 임베딩 저장
cluster_labels_list = []
embeddings_list = []

for label, embeddings in cluster_embeddings.items():
    for embedding in embeddings:
        cluster_labels_list.append(label)
        embeddings_list.append(embedding.numpy())

# DataFrame으로 변환
embedding_df = pd.DataFrame(embeddings_list)
embedding_df['클러스터'] = cluster_labels_list

# 최종 결과 CSV로 저장
embedding_df.to_csv('cluster_embeddings.csv', index=False, encoding='utf-8-sig')

print("KOBERT 라벨과 임베딩이 'cluster_embeddings.csv' 파일에 저장되었습니다.")

KOBERT 라벨과 임베딩이 'cluster_embeddings.csv' 파일에 저장되었습니다.


In [22]:
print(embedding_df.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', '클러스터',
       'similarity'],
      dtype='object', length=770)


In [21]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# DB에서 클러스터 임베딩 로드 (CSV 파일을 사용하여 시뮬레이션)
embedding_df = pd.read_csv(r'C:\Users\user\Documents\no-f\241030\df_inheritance_keyword.csv')

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 인코딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 문장 임베딩

# 사용자 입력 문장
user_input = "세법"
user_embedding = encode_sentence(user_input)

# DB에서 모든 문서 임베딩 가져오기
all_embeddings = embedding_df.drop(columns='클러스터').values  # 클러스터 열 제외
all_embeddings_tensor = torch.tensor(all_embeddings)

# 유사도 계산
similarity_scores = cosine_similarity(user_embedding.numpy(), all_embeddings_tensor.numpy()).flatten()

# 유사도에 따라 문서 정렬
embedding_df['similarity'] = similarity_scores
sorted_documents = embedding_df.sort_values(by='similarity', ascending=False)

# 가장 유사한 문서 상위 N개 선택
top_n = 10  # 보여줄 문서 개수
recommended_documents = sorted_documents.head(top_n)

# 추천 문서 출력
print("세법과 관련된 문서:")
for idx, row in recommended_documents.iterrows():
    print(f"클러스터: {row['클러스터']}, 유사도: {row['similarity']:.4f}, 내용: {row['참조조문']}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


세법과 관련된 문서:


KeyError: '참조조문'

In [30]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# 1. 데이터프레임 로드
df = pd.read_csv(r'C:\Users\user\Documents\no-f\241030\df_inheritance_keyword.csv')  # 데이터 파일 경로를 입력하세요.

# 2. KoBERT 모델 및 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 3. 사용자 입력 받기
user_input = input("클러스터 키워드를 입력하세요 (예: 세법): ")

# 4. 클러스터키워드에 해당하는 데이터 필터링
filtered_df = df[df['키워드'] == user_input]

# 5. 추천 참조조문 추출
if not filtered_df.empty:
    # 사용자 입력 벡터화
    user_vector = model(**tokenizer(user_input, return_tensors='pt', truncation=True, padding=True)).last_hidden_state.mean(dim=1)

    similarity_scores = []
    
    for index, row in filtered_df.iterrows():
        if pd.notnull(row['판결요지']):  # 판결요지나 다른 텍스트를 기준으로 유사도 계산
            case_vector = model(**tokenizer(row['판결요지'], return_tensors='pt', truncation=True, padding=True)).last_hidden_state.mean(dim=1)
            # 코사인 유사도 계산
            similarity = torch.nn.functional.cosine_similarity(user_vector, case_vector).item()
            similarity_scores.append((similarity, row['참조조문']))

    # 6. 유사도 기준으로 정렬 및 추천
    similarity_scores.sort(reverse=True, key=lambda x: x[0])
    recommended_references = [ref for score, ref in similarity_scores if score > 0.5]  # 유사도 기준

    # 7. 추천 출력
    if recommended_references:
        print("추천 참조조문:")
        for ref in recommended_references:
            print(ref)
    else:
        print("해당 키워드와 관련된 참조조문이 없습니다.")
else:
    print("해당 키워드와 일치하는 데이터가 없습니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


해당 키워드와 일치하는 데이터가 없습니다.


In [47]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 데이터프레임 로드
df_inheritance = pd.read_csv('df_inheritance_keyword.csv')

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# 판결요지 임베딩 생성
df_inheritance['임베딩'] = df_inheritance['판결요지'].apply(lambda x: encode_sentence(x).numpy() if pd.notnull(x) else None)

# 사용자 입력 문장
user_input = "세법"
user_embedding = encode_sentence(user_input)

# 유사도 계산 및 정렬
all_embeddings = np.vstack(df_inheritance['임베딩'].dropna())
similarity_scores = cosine_similarity(user_embedding.numpy(), all_embeddings).flatten()
df_inheritance['유사도'] = similarity_scores

# 가장 유사한 문서 상위 N개 선택
top_n = 10
recommended_documents = df_inheritance.sort_values(by='유사도', ascending=False).head(top_n)

print("세법과 관련된 문서:")
print(recommended_documents[['판례정보일련번호', '판결요지', '유사도']])


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


ValueError: Length of values (2930) does not match length of index (4162)